In [11]:
# ********************************************************************************
# Copyright (c) 2024 Contributors to the Eclipse Foundation
#
# See the NOTICE file(s) distributed with this work for additional
# information regarding copyright ownership.
#
# This program and the accompanying materials are made available under the
# terms of the Apache License, Version 2.0 which is available at
# https://www.apache.org/licenses/LICENSE-2.0.
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS, WITHOUT
# WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the
# License for the specific language governing permissions and limitations
# under the License.
#
# SPDX-License-Identifier: Apache-2.0
# ********************************************************************************

import requests
import json
import logging
import importlib
import ghapi
import config

importlib.reload(ghapi)
importlib.reload(config)

# create a re-usable session object with the user creds in-built
gh_session = requests.Session()
gh_session.auth = (config.username, config.token)

logger = logging.getLogger(__name__)
logging.basicConfig(level=config.loglevel)


In [12]:
# Get the teams in the GitHub Org (public or private ones you are member of):
team_slugs =  ghapi.getTeamSlugs(gh_session, config.organizationName)

In [13]:
# Get the team members:
# For each member (login) we collect the infos about the team memberships.

# Links:
# https://docs.github.com/en/rest/teams/members?apiVersion=2022-11-28#list-team-members
# https://api.github.com/orgs/ORG/teams/TEAM_SLUG/members

memberships = {}
elements = ''

# Default team membership (set all teams to false)
membership_default = {}
for ts in team_slugs:
    membership_default.update({ts: False})

for team_slug in team_slugs:

    members_url =  'https://api.github.com/orgs/{ORG}/teams/{TEAM_SLUG}/members'.format(ORG=config.organizationName, TEAM_SLUG=team_slug)
    elements = ghapi.fetchAll(gh_session, members_url)
    # Request user info: name, mail (if publicly available in the profile)
    team_members = [{"login": m["login"], "id": m['id'], "profile": m['html_url']} for m in elements]

    for tm in team_members:

        if tm["login"] not in memberships.keys():
            # Add default team membership items:
            tm.update(membership_default)
            memberships.update({tm["login"]: tm})

        (memberships.get(tm['login'])).update({team_slug: True})

# print(memberships)


In [14]:
import requests
import pandas
importlib.reload(ghapi)
importlib.reload(config)

headers = {"Authorization": "token " + config.token}

def run_query(query, variables):
    response = requests.post('https://api.github.com/graphql', json={'query': query, 'variables': variables}, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception("Query failed to run by returning code of {}. {}".format(response.status_code, query))


toDate, toDateCSV, fromDate, fromDateCSV = ghapi.contribution_interval(config.contrib_nr_of_months)

# The GraphQL query itself defined as a multi-line string.
query = """
query($organizationName: String!, $organizationID: ID!, $cursorID: String, $fromDate: DateTime!, $toDate: DateTime!) {
  organization(login: $organizationName) {
    membersWithRole(first: 25, after: $cursorID) {
      nodes {
        login
        contributionsCollection(organizationID: $organizationID, from: $fromDate, to: $toDate) {
          hasAnyContributions
          hasActivityInThePast
          totalCommitContributions
          totalIssueContributions
          totalPullRequestContributions
          totalPullRequestReviewContributions
          totalRepositoriesWithContributedIssues
          totalRepositoriesWithContributedCommits
          totalRepositoriesWithContributedPullRequests
          totalRepositoriesWithContributedPullRequestReviews
        }
      }
      pageInfo {
        hasNextPage
        endCursor
      }
    }
  }
}
"""

contribs = []
hasNextPage = True
cursorID = None

while hasNextPage:

  variables = {
    "organizationName": config.organizationName,
    "organizationID": config.organizationID,
    "fromDate": fromDate,
    "toDate": toDate,
    "cursorID": cursorID
  }

  result = run_query(query, variables) # Execute the query
  # print(result)

  elements = result['data']['organization']['membersWithRole']
  nodes = elements['nodes']

  cursorID = elements['pageInfo']['endCursor']
  hasNextPage = elements['pageInfo']['hasNextPage']
  # for tests only:
  # hasNextPage = False

  for node in nodes:
      login = node['login']

      # from API: https://docs.github.com/en/graphql/reference/objects#organizationmemberconnection
      # hasAnyContributions: Determine if there are any contributions in this collection.
      # hasActivityInThePast: Does the user have any more activity in the timeline that occurred prior to the collection's time range?
      # totalCommitContributions: How many commits were made by the user in this time span.
      # totalIssueContributions: How many issues the user opened.
      # totalPullRequestContributions: How many pull requests the user opened.
      # totalPullRequestReviewContributions: How many pull request reviews the user left.
      # totalRepositoriesWithContributedIssues: How many different repositories the user opened issues in.
      # totalRepositoriesWithContributedCommits: How many different repositories the user committed to.
      # totalRepositoriesWithContributedPullRequests: How many different repositories the user opened pull requests in.
      # totalRepositoriesWithContributedPullRequestReviews: How many different repositories the user left pull request reviews in.

      cb = [{"GitHub login (" + fromDateCSV+ ","  + toDateCSV + ")": login,
              'totalCommitContributions': node['contributionsCollection']['totalCommitContributions'],
              'totalIssueContributions': node['contributionsCollection']['totalIssueContributions'],
              'totalPullRequestContributions': node['contributionsCollection']['totalPullRequestContributions'],
              'totalPullRequestReviewContributions': node['contributionsCollection']['totalPullRequestReviewContributions'],
              'sum_contributions': node['contributionsCollection']['totalCommitContributions']+ node['contributionsCollection']['totalIssueContributions']+node['contributionsCollection']['totalPullRequestContributions']+node['contributionsCollection']['totalPullRequestReviewContributions']
              }]
      # Add membership info:
      membership_info = {}
      if login in memberships.keys():
          membership_info = memberships.get(login)
      cb[0].update(membership_info)

      contribs.extend(cb)


INFO:ghapi:Contribution interval requested: 2024-01-12 - 2024-07-12


In [15]:
df = pandas.DataFrame.from_dict(contribs)
df.to_csv(toDateCSV+'_'+config.fname_github_contributions, sep=';', index=False, encoding='utf-8')